In [1]:
!pip -q install folium
!pip install streamlit streamlit-folium folium pandas
!npm install -g localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 137.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.7 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇
added 22 packages in 2s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇

In [2]:
# 나눔고딕 폰트 설치 및 설정
!apt-get update -qq
!apt-get install fonts-nanum -qq
!fc-cache -fv
!rm ~/.cache/matplotlib -rf

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/nanum: skipping, looped dire

2025-07-02 07:33:48.269 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [28]:
%%writefile school.py
# 파일명 예: app.py
import streamlit as st
import pandas as pd
import folium
from folium.plugins import HeatMap
from streamlit_folium import folium_static


import pandas as pd
import folium
import matplotlib.pyplot as plt
from folium.plugins import HeatMap
import matplotlib.font_manager as fm

# 폰트 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False
# 1. 데이터 로드
df = pd.read_excel('/content/학교주소좌표.xlsx')
df.columns = ['학교명', '주소', '경도', '위도']

# 2. 시군구 추출 함수 정의
def extract_city_district(address):
    parts = address.split()
    return " ".join(parts[:2]) if len(parts) >= 2 else address

df['시군구'] = df['주소'].apply(extract_city_district)

# 3. 시군구별 학교 수 집계
district_counts = df['시군구'].value_counts().sort_values(ascending=False)


st.title(" 시군구별 대학 수 시각화")


# ───────────────────────────────
#  시군구별 학교 수 막대 그래프
st.subheader(" 시군구별 대학 수 (상위 20개)")
fig1, ax1 = plt.subplots(figsize=(14, 8))
district_counts[:20].plot(kind='bar')
plt.title('시군구별  대학 수 (상위 20개)')
plt.xlabel('시군구')
plt.ylabel('대학 수')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
#plt.show()
st.pyplot(fig1)



# ───────────────────────────────
# 시군구별 파이 차트 (상위 10개만)
st.subheader(" 상위 10개 시군구 대학 분포 비율")
fig2, ax2 = plt.subplots(figsize=(8, 8))
district_counts[:10].plot(kind='pie', autopct='%1.1f%%', startangle=140)
plt.title('상위 10개 시군구 대학 분포 비율')
plt.ylabel('')
plt.tight_layout()
#plt.show()
st.pyplot(fig2)

# ───────────────────────────────
#  지도 히트맵 시각화
#map_center = [df['위도'].mean(), df['경도'].mean()]
map_center = [36.5, 127.8]
heatmap = folium.Map(location=map_center, zoom_start=7)

heat_data = df[['위도', '경도']].values.tolist()
HeatMap(heat_data).add_to(heatmap)

# 사용자 입력 받기 (검색어)
search_school = st.text_input("검색할 학교 이름을 입력하세요 (예: 백석문화대학교)", "")

if search_school:
    filtered = df[df['학교명'].str.contains(search_school, case=False)]

    if not filtered.empty:
        for i in filtered.index:
            name = filtered.loc[i, '학교명']
            lat = filtered.loc[i, '위도']
            lon = filtered.loc[i, '경도']

            folium.Marker(
                location=[lat, lon],
                tooltip=name,
                popup=folium.Popup(name, max_width=250),
                icon=folium.Icon(color='red',  icon='school', prefix='fa')
            ).add_to(heatmap)
    else:
        st.warning(f"'{search_school}'로 검색된 학교가 없습니다.")


# 지도 출력
folium_static(heatmap)

Overwriting school.py


In [12]:
!wget -q -O - ipv4.icanhazip.com

34.55.243.156


In [ ]:
!streamlit run school.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.55.243.156:8501

⠼⠴⠦⠧⠇your url is: https://ten-eyes-kick.loca.lt
/content/school.py:97: DeprecationWarning: 
folium_static is deprecated and will be removed in a future release, or
simply replaced with with st_folium which always passes
returned_objects=[] to the component.
Please try using st_folium instead, and
post an issue at https://github.com/randyzwitch/streamlit-folium/issues
if you experience issues with st_folium.

  folium_static(heatmap)
/content/school.py:97: DeprecationWarning: 
folium_static is deprecated and will be removed in a future release, or
simply replaced with with st_folium which always passes
returned_objects=[] to the component.
Please try using st_folium instead, and
post an issue at https://github.com/randyzwitch/streamlit-folium/issues
if you experience issues with st_folium.

  folium_static(heat